<a href="https://colab.research.google.com/github/alermar69/HELP/blob/master/Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import

In [ ]:
import numpy as np
import pandas as pd

from tensorflow.keras.models import Sequential #НС прямого распространения
from tensorflow.keras.layers import Dense, Activation, Dropout, BatchNormalization #Основные слои
#Базовые слои для счёрточных сетей
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam, Adadelta # оптимизаторы
from tensorflow.keras import utils #Утилиты для to_categorical

import matplotlib.pyplot as plt
import pylab # Модуль для построения графиков
from mpl_toolkits.mplot3d import Axes3D # Модуль для трехмерной графики
from google.colab import files #Для загрузки своей картинки
from tensorflow.keras.preprocessing import image #Для отрисовки изображения
from PIL import Image #Отрисовка изображений
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator # работа с изображениями

import random #Для генерации случайных чисел 
import math # Для округления
import os #Для работы с файлами 
from google.colab import drive # подключем диск
from google.colab import files #Для загрузки своей картинки

from tensorflow.keras.datasets import mnist
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.datasets import cifar100

## Base

### MNIST

In [ ]:
(x_train_org, y_train_org), (x_test_org, y_test_org) = mnist.load_data() #Загрузка данных Mnist
plt.imshow(5, cmap='viridis') #Отрисовка картинки

#Меняем формат входных картинок с 28х28 на 784х1
x_train = x_train_org.reshape(60000, 784)
x_test = x_test_org.reshape(10000, 784)

#Нормализуем входные картинки
x_train = x_train.astype('float32') # преобразовываем x_train в тип float (цифры с плавающей точкой)
x_train = x_train / 255 # делим на 255, чтобы диапазон был от 0 до 1
x_test = x_test.astype('float32')
x_test = x_test / 255

# Преобразуем ответы в формат one_hot_encoding
y_train = utils.to_categorical(y_train_org, 10)
y_test = utils.to_categorical(y_test_org, 10)

# Создание нейронной сети
model = Sequential() # Создаём сеть прямого распространения
model.add(Dense(800, input_dim=784, activation="relu")) # Добавляем полносвязный слой на 800 нейронов с relu-активацией
model.add(Dense(400, activation="relu")) # Добавляем полносвязный слой на 400 нейронов с relu-активацией
model.add(Dense(10, activation="softmax")) # Добавляем полносвязный слой на 10 нейронов с softmax-активацией

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"]) # Компилируем модель
print(model.summary()) #Вывод структуры модели 

# Обучение нейронной сети
model.fit(x_train, y_train, batch_size=128, epochs=15, verbose=1, validation_split=0.2)

model.save_weights('model.h5')
model.load_weights('model.h5')

# Распознавание рукописных цифр

#Выбираем нужную картинку из тестовой выборки
x = x_test[5]
#Добавляем одну размерность в конце, чтобы нейронка могла распознать пример
#Потому что нейронка принимает именно массив примеров для распознавания
#Мы делаем массив из одного примера
x = np.expand_dims(x, axis=0)
prediction = model.predict(x) #Распознаём наш пример
prediction = np.argmax(prediction) # Получаем индекс самого большого элемента (это итоговая цифра, которую распознала сеть)


# Загружаем картинку сделанную в графическом редакторе
example = image.load_img('/content/4_.png', target_size=(28, 28), color_mode = 'grayscale') 
# Нормализуем данные
example = image.img_to_array(example) # преобразуем изображение в numpy-массив
example = example.reshape(1,784)
example = np.where(example > 100, 255, example) # убираем шум
example = example.astype('float32')
example = 1- example/255 # Делаем инверсию, чтобы было не черное на белом, а белая цифра на черном фоне
#Распознаём наш пример
pred_example = model_sample.predict(example) 


### Создание модели

In [ ]:
#Функция пересоздаёт пустую сеть
def createModel():
  model = Sequential()
  model.add(Dense(800, input_dim=784, activation="relu"))
  model.add(Dropout(0.2)) #20% нейронов будут выключены
  model.add(BatchNormalization()) #Добавляем слой пакетной нормализации
  model.add(Dense(25,  activation="tanh"))
  model.add(Dense(10, activation="softmax"))
  
  model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
  model.compile(loss="categorical_crossentropy", optimizer=Adam(lr=0.006), metrics=["accuracy"])

  return model

### Варианты создания проверочной выборки

In [ ]:
#validation_split
model.fit(x_train, y_train, batch_size=200, epochs=3, verbose=1,
          validation_split=0.2) #Указываем 20% случайных примеров для проверочной выборки

#validation_data
n_val = 10000 #Указываем 10.000 примеров в проверочную выборку
x_len = x_train.shape[0] #Запоминаем размер всей выборки целиком

model.fit(x_train[:x_len-n_val], #Берём все до индекса 60.000 - 10.000 = 50.000
          y_train[:x_len-n_val], 
          batch_size=200, epochs=3,
          #Прямо указываем данные для проверочной выборки
          #Берём от индекса 50.000 до конца = 10.000 примеров
          validation_data=(x_train[x_len-n_val:], y_train[x_len-n_val:]))

#sklearn train_test_split
from sklearn.model_selection import train_test_split
x_train_new, x_val, y_train_new, y_val = train_test_split(x_train, y_train, test_size=0.2)
model.fit(x_train_new, y_train_new, batch_size=200, epochs=3,
          validation_data=(x_val, y_val)) #Пряму указываем проверочную выборку
          

### Проверяем качество обучения на тестовом наборе данных

In [ ]:
scores = model.evaluate(x_test, y_test, verbose=1)
#scores состоит из двух знанчений
#scores[0] - loss сети на тестовой выборке
#scores[1] - процент правильно распознанных примеров на тестовой выборке

### Визуализация качества обучения

In [ ]:
history = model.fit(x_train,y_train,batch_size=200, epochs=20, validation_split=0.2, verbose=1)

#Ключи в структуре history
print(history.history.keys()) #dict_keys(['loss', 'acc', 'val_loss', 'val_acc'])

#Значения loss на обучающей выборке по эпохам
history.history['loss']

#Значения точности распознавания на проверочной выборке по эпохам
val_acc = history.history['val_acc']

#Выводим графики точности
plt.plot(history.history['acc'], label='Доля верных ответов на обучающем наборе')
plt.plot(history.history['val_acc'], label='Доля верных ответов на проверочном наборе')
plt.xlabel('Эпоха обучения')
plt.ylabel('Доля верных ответов')
plt.legend()

#Выводим графики ошибки
plt.plot(history.history['loss'], label='Ошибка на обучающем наборе')
plt.plot(history.history['val_loss'], label='Ошибка на проверочном наборе')
plt.xlabel('Эпоха обучения')
plt.ylabel('Ошибка')
plt.legend()

## CNN

In [ ]:
batch_size = 128 

model = Sequential()
model.add(BatchNormalization(input_shape=(28, 28, 1)))
model.add(Conv2D(32, 3, padding='same', activation='relu'))
model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(10, activation='softmax'))

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

history = model.fit(x_train, y_train, verbose=1,
                    batch_size=batch_size, epochs=15,
                    validation_data=(x_test, y_test))

plt.plot(history.history['accuracy'], label='Доля верных ответов на обучающем наборе')
plt.plot(history.history['val_accuracy'], label='Доля верных ответов на проверочном наборе')
plt.xlabel('Эпоха обучения')
plt.ylabel('Доля верных ответов')
plt.legend()

model.summary()